In [1]:
import fml_lib
import numpy as np
import pandas as pd
import scipy.stats as ss
import os.path
from math import sqrt, exp, log
import matplotlib.pyplot as plt
from mp_engine import mpPandasObj
from datetime import datetime

In [2]:
dollar_bars = pd.read_csv("USA500IDXUSD_hourly_bars.csv", parse_dates=[['Date', 'Timestamp']])
dollar_bars = dollar_bars.set_index("Date_Timestamp")
dollar_bars = dollar_bars.dropna()
close = dollar_bars['Close']
ffd_close = np.log(dollar_bars[['Close']]).cumsum()
print(ffd_close)


                             Close
Date_Timestamp                    
2019-04-17 00:00:00       7.975176
2019-04-17 01:00:00      15.951658
2019-04-17 02:00:00      23.927521
2019-04-17 03:00:00      31.903144
2019-04-17 04:00:00      39.878664
...                            ...
2024-04-16 18:00:00  243348.708861
2024-04-16 19:00:00  243357.236147
2024-04-16 20:00:00  243365.763374
2024-04-16 22:00:00  243374.292003
2024-04-16 23:00:00  243382.822105

[29496 rows x 1 columns]


C:\Users\David\AppData\Local\Temp\ipykernel_95904\128212720.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  dollar_bars = pd.read_csv("USA500IDXUSD_hourly_bars.csv", parse_dates=[['Date', 'Timestamp']])


In [3]:
def plotMinFFD():
    from statsmodels.tsa.stattools import adfuller
    import matplotlib.pyplot as mpl
    out=pd.DataFrame(columns=['adfStat','pVal','lags','nObs','95% conf','corr'])
    df0=dollar_bars
    for d in np.linspace(0.9,1,31):
        df1=np.cumsum(np.log(df0[['Close']])).resample('1D').last() # downcast to daily obs
        df2=fml_lib.fracDiff_FFD(df1,d,thres=1E-3)
        corr=np.corrcoef(df1.loc[df2.index,'Close'],df2['Close'])[0,1]
        df2=adfuller(df2['Close'],maxlag=1,regression='c',autolag=None)
       
        out.loc[d]=list(df2[:4])+[df2[4]['5%']]+[corr] # with critical value
    out.to_csv('./_testMinFFD.csv')
    out[['adfStat','corr']].plot(secondary_y='adfStat')
    mpl.axhline(out['95% conf'].mean(),linewidth=1,color='r',linestyle='dotted')
    mpl.savefig('./_testMinFFD.png')
    return

In [4]:
test_ffd = fml_lib.fracDiff_FFD(ffd_close,0.97)
ffd_std = test_ffd.std()[0]
#print(ffd_std)
daily_volatility = fml_lib.getDailyVol(test_ffd.Close,span0=100)

c:\Users\David\Classes\ECE479\fml_lib.py:284: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  seriesF,df_=series[[name]].fillna(method='ffill').dropna(),pd.Series()


58


C:\Users\David\AppData\Local\Temp\ipykernel_95904\3059649216.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ffd_std = test_ffd.std()[0]


In [5]:
print(test_ffd)
print(daily_volatility)

                          Close
2019-04-22 12:00:00    9.156934
2019-04-22 13:00:00    9.164251
2019-04-22 14:00:00    9.169456
2019-04-22 15:00:00    9.174572
2019-04-22 16:00:00    9.179339
...                         ...
2024-04-16 18:00:00  154.018949
2024-04-16 19:00:00  154.019507
2024-04-16 20:00:00  154.024411
2024-04-16 22:00:00  154.030827
2024-04-16 23:00:00  154.037373

[29438 rows x 1 columns]
2019-04-23 13:00:00         NaN
2019-04-23 14:00:00    0.000056
2019-04-23 15:00:00    0.000185
2019-04-23 16:00:00    0.000196
2019-04-23 17:00:00    0.000185
                         ...   
2024-04-16 18:00:00    0.000181
2024-04-16 19:00:00    0.000180
2024-04-16 20:00:00    0.000179
2024-04-16 22:00:00    0.000178
2024-04-16 23:00:00    0.000177
Name: Close, Length: 29414, dtype: float64


In [6]:

if os.path.exists("./t_events.csv"):
    t_events = pd.read_csv("t_events.csv", index_col=[0])
    t_events = t_events.index
else:
    t_events = fml_lib.getTEvents(test_ffd.Close, 2*daily_volatility)
    t_events_df = pd.DataFrame(index= t_events)
    t_events_df.to_csv("./t_events_csv", index=True)

c:\Users\David\Classes\ECE479\fml_lib.py:115: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if sNeg<-h.iloc[h.index.get_indexer([i], method='nearest')][0]:
c:\Users\David\Classes\ECE479\fml_lib.py:118: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif sPos>h.iloc[h.index.get_indexer([i], method='nearest')][0]:


In [7]:
print(t_events)

DatetimeIndex(['2019-04-23 14:00:00', '2019-04-23 15:00:00',
               '2019-04-23 16:00:00', '2019-04-23 17:00:00',
               '2019-04-23 18:00:00', '2019-04-23 19:00:00',
               '2019-04-23 20:00:00', '2019-04-23 22:00:00',
               '2019-04-23 23:00:00', '2019-04-24 00:00:00',
               ...
               '2024-04-16 13:00:00', '2024-04-16 14:00:00',
               '2024-04-16 15:00:00', '2024-04-16 16:00:00',
               '2024-04-16 17:00:00', '2024-04-16 18:00:00',
               '2024-04-16 19:00:00', '2024-04-16 20:00:00',
               '2024-04-16 22:00:00', '2024-04-16 23:00:00'],
              dtype='datetime64[ns]', length=28277, freq=None)


In [8]:
test_ffd['original_close'] = close.loc[t_events]
test_ffd['Close'] = test_ffd['Close'].loc[t_events]
test_ffd = test_ffd.dropna()
print(test_ffd)
#adds the vertical barrier num_days out from the time of the event
numDays = 5
t1=test_ffd.index.searchsorted(t_events+pd.Timedelta(days=numDays))
t1=t1[t1<test_ffd.shape[0]]
t1=pd.Series(test_ffd.index[t1],index=t_events[:t1.shape[0]]) # NaNs at end

                          Close  original_close
2019-04-23 14:00:00    9.284123        2920.470
2019-04-23 15:00:00    9.292608        2931.169
2019-04-23 16:00:00    9.298190        2933.170
2019-04-23 17:00:00    9.302872        2932.669
2019-04-23 18:00:00    9.308194        2934.170
...                         ...             ...
2024-04-16 18:00:00  154.018949        5073.976
2024-04-16 19:00:00  154.019507        5050.723
2024-04-16 20:00:00  154.024411        5050.421
2024-04-16 22:00:00  154.030827        5057.506
2024-04-16 23:00:00  154.037373        5064.960

[28277 rows x 2 columns]


In [9]:
print(t1)

2019-04-23 14:00:00   2019-04-28 22:00:00
2019-04-23 15:00:00   2019-04-28 22:00:00
2019-04-23 16:00:00   2019-04-28 22:00:00
2019-04-23 17:00:00   2019-04-28 22:00:00
2019-04-23 18:00:00   2019-04-28 22:00:00
                              ...        
2024-04-11 18:00:00   2024-04-16 18:00:00
2024-04-11 19:00:00   2024-04-16 19:00:00
2024-04-11 20:00:00   2024-04-16 20:00:00
2024-04-11 22:00:00   2024-04-16 22:00:00
2024-04-11 23:00:00   2024-04-16 23:00:00
Length: 28209, dtype: datetime64[ns]


In [10]:
min_ret= 0.005
#trgt = pd.Series(0.01, index=t_events)
trgt = 2*daily_volatility
'''
if os.path.exists("./events.csv"):
    events = pd.read_csv("events.csv", index_col=[0])
    events = events.dropna()
else:
    events = fml_lib.getEvents(close, t_events, 0.02,np.log(trgt) , np.log(min_ret),t1)
    events.to_csv("events.csv")
'''
events = fml_lib.getEvents(test_ffd['original_close'], t_events, 0.02,trgt , min_ret,t1)
events.to_csv("events.csv")

datetime64[ns]


In [11]:
print(events)

                                     t1      trgt  side
2019-04-29 00:00:00 2019-04-29 02:00:00  0.005987   1.0
2019-04-29 02:00:00 2019-04-29 06:00:00  0.007055   1.0
2019-04-29 04:00:00 2019-04-29 06:00:00  0.007639   1.0
2019-04-29 06:00:00 2019-04-29 08:00:00  0.007932   1.0
2019-04-29 08:00:00 2019-04-29 11:00:00  0.008030   1.0
...                                 ...       ...   ...
2019-06-24 13:00:00 2019-06-24 14:00:00  0.005399   1.0
2019-06-24 14:00:00 2019-06-24 16:00:00  0.005354   1.0
2019-06-24 16:00:00 2019-06-24 18:00:00  0.005253   1.0
2019-06-24 18:00:00 2019-06-24 20:00:00  0.005150   1.0
2019-06-24 20:00:00 2019-06-24 23:00:00  0.005056   1.0

[237 rows x 3 columns]


In [12]:
close = test_ffd['original_close']
out = fml_lib.getBins(events, close, numDays)
print(out)

                          ret  bin
2019-04-29 00:00:00  0.000442  1.0
2019-04-29 02:00:00 -0.000170 -1.0
2019-04-29 04:00:00 -0.000170 -1.0
2019-04-29 06:00:00  0.000646  1.0
2019-04-29 08:00:00 -0.001428 -1.0
...                       ...  ...
2019-06-24 13:00:00  0.000915  1.0
2019-06-24 14:00:00 -0.000169 -1.0
2019-06-24 16:00:00 -0.000847 -1.0
2019-06-24 18:00:00 -0.000373 -1.0
2019-06-24 20:00:00  0.000101  1.0

[237 rows x 2 columns]


In [13]:
print(events)

                                     t1      trgt  side
2019-04-29 00:00:00 2019-04-29 02:00:00  0.005987   1.0
2019-04-29 02:00:00 2019-04-29 06:00:00  0.007055   1.0
2019-04-29 04:00:00 2019-04-29 06:00:00  0.007639   1.0
2019-04-29 06:00:00 2019-04-29 08:00:00  0.007932   1.0
2019-04-29 08:00:00 2019-04-29 11:00:00  0.008030   1.0
...                                 ...       ...   ...
2019-06-24 13:00:00 2019-06-24 14:00:00  0.005399   1.0
2019-06-24 14:00:00 2019-06-24 16:00:00  0.005354   1.0
2019-06-24 16:00:00 2019-06-24 18:00:00  0.005253   1.0
2019-06-24 18:00:00 2019-06-24 20:00:00  0.005150   1.0
2019-06-24 20:00:00 2019-06-24 23:00:00  0.005056   1.0

[237 rows x 3 columns]


Want to find sample weights for use in the boosting fit

In [14]:
temp = list(set(pd.to_datetime(events.t1.index)) & set(pd.to_datetime(events.index)) )

In [15]:
#test_co_events = mpPandasObj(fml_lib.mpNumCoEvents,('molecule',events.index),24, closeIdx=close.index,t1=events['t1'])

if os.path.exists("./t_events.csv"):  
    numCoEvents = pd.read_csv("./co_events.csv", index_col=[0])
else:
    numCoEvents = fml_lib.get_co_events(close,events)
    numCoEvents.to_csv("./co_events.csv", index=True)

2019-04-29 00:00:00    1
2019-04-29 02:00:00    2
2019-04-29 04:00:00    2
2019-04-29 06:00:00    3
2019-04-29 08:00:00    2
                      ..
2019-06-24 14:00:00    2
2019-06-24 16:00:00    2
2019-06-24 18:00:00    2
2019-06-24 20:00:00    2
2019-06-24 23:00:00    1
Length: 627, dtype: int64


In [16]:

old_index = numCoEvents.index
old_co = numCoEvents.values

In [17]:
numCoEvents=numCoEvents.loc[~numCoEvents.index.duplicated(keep='last')]
numCoEvents=numCoEvents.reindex(close.index,fill_value=0)
numCoEvents.loc[old_index] = old_co
print(numCoEvents.loc[old_index])
print(numCoEvents)

2019-04-29 00:00:00    1
2019-04-29 02:00:00    2
2019-04-29 04:00:00    2
2019-04-29 06:00:00    3
2019-04-29 08:00:00    2
                      ..
2019-06-24 14:00:00    2
2019-06-24 16:00:00    2
2019-06-24 18:00:00    2
2019-06-24 20:00:00    2
2019-06-24 23:00:00    1
Length: 627, dtype: int64
2019-04-23 14:00:00    0
2019-04-23 15:00:00    0
2019-04-23 16:00:00    0
2019-04-23 17:00:00    0
2019-04-23 18:00:00    0
                      ..
2024-04-16 18:00:00    0
2024-04-16 19:00:00    0
2024-04-16 20:00:00    0
2024-04-16 22:00:00    0
2024-04-16 23:00:00    0
Length: 28277, dtype: int64


In [18]:
numThreads = 12

out['tW']=mpPandasObj(fml_lib.mpSampleTW,('molecule',events.index),numThreads, t1=events['t1'],numCoEvents=numCoEvents)

pool made
variables initialized
in loop
Reporting
in loop
Reporting
in loop
Reporting


in loop
Reporting
in loop
Reporting
in loop
Reporting
in loop
Reporting
in loop
Reporting


in loop
Reporting
in loop
Reporting
in loop
Reporting
in loop
Reporting


2024-04-24 03:15:23.017737 100.0% mpSampleTW done after 0.13 minutes. Remaining 0.0 minutes..
c:\Users\David\Classes\ECE479\mp_engine.py:127: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df0=df0._append(i)


In [19]:

out['w'] = mpPandasObj(fml_lib.mpSampleW,('molecule',events.index),numThreads, t1=events['t1'],numCoEvents=numCoEvents,close=close)
print(out['w'])

pool made
variables initialized
in loop
Reporting
in loop
Reporting
in loop
Reporting


in loop
Reporting
in loop
Reporting


in loop
Reporting


in loop
Reporting
in loop
Reporting
in loop
Reporting
in loop
Reporting


in loop
Reporting


in loop
Reporting


2024-04-24 03:15:31.984738 100.0% mpSampleW done after 0.13 minutes. Remaining 0.0 minutes.


2019-04-29 00:00:00    0.000221
2019-04-29 02:00:00    0.000164
2019-04-29 04:00:00   -0.000057
2019-04-29 06:00:00    0.000266
2019-04-29 08:00:00   -0.000392
                         ...   
2019-06-24 13:00:00   -0.000034
2019-06-24 14:00:00    0.000373
2019-06-24 16:00:00   -0.000508
2019-06-24 18:00:00   -0.000610
2019-06-24 20:00:00   -0.000085
Name: w, Length: 237, dtype: float64


c:\Users\David\Classes\ECE479\mp_engine.py:127: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  df0=df0._append(i)


In [20]:
print(out)

                          ret  bin        tW         w
2019-04-29 00:00:00  0.000442  1.0  0.750000  0.000221
2019-04-29 02:00:00 -0.000170 -1.0  0.444444  0.000164
2019-04-29 04:00:00 -0.000170 -1.0  0.416667 -0.000057
2019-04-29 06:00:00  0.000646  1.0  0.416667  0.000266
2019-04-29 08:00:00 -0.001428 -1.0  0.500000 -0.000392
...                       ...  ...       ...       ...
2019-06-24 13:00:00  0.000915  1.0  0.500000 -0.000034
2019-06-24 14:00:00 -0.000169 -1.0  0.500000  0.000373
2019-06-24 16:00:00 -0.000847 -1.0  0.500000 -0.000508
2019-06-24 18:00:00 -0.000373 -1.0  0.500000 -0.000610
2019-06-24 20:00:00  0.000101  1.0  0.750000 -0.000085

[237 rows x 4 columns]


In [21]:
print(events['t1'])

2019-04-29 00:00:00   2019-04-29 02:00:00
2019-04-29 02:00:00   2019-04-29 06:00:00
2019-04-29 04:00:00   2019-04-29 06:00:00
2019-04-29 06:00:00   2019-04-29 08:00:00
2019-04-29 08:00:00   2019-04-29 11:00:00
                              ...        
2019-06-24 13:00:00   2019-06-24 14:00:00
2019-06-24 14:00:00   2019-06-24 16:00:00
2019-06-24 16:00:00   2019-06-24 18:00:00
2019-06-24 18:00:00   2019-06-24 20:00:00
2019-06-24 20:00:00   2019-06-24 23:00:00
Name: t1, Length: 237, dtype: datetime64[ns]


In [22]:
print(test_ffd.loc[out.index])

                         Close  original_close
2019-04-29 00:00:00   9.660999        2938.670
2019-04-29 02:00:00   9.670972        2939.969
2019-04-29 04:00:00   9.680507        2939.970
2019-04-29 06:00:00   9.689869        2939.470
2019-04-29 08:00:00   9.700027        2941.370
...                        ...             ...
2019-06-24 13:00:00  14.047657        2948.370
2019-06-24 14:00:00  14.053285        2951.069
2019-06-24 16:00:00  14.062623        2950.569
2019-06-24 18:00:00  14.071227        2948.070
2019-06-24 20:00:00  14.080293        2946.970

[237 rows x 2 columns]


In [23]:
for item in events['t1'].items():
    print(item)

(Timestamp('2019-04-29 00:00:00'), Timestamp('2019-04-29 02:00:00'))
(Timestamp('2019-04-29 02:00:00'), Timestamp('2019-04-29 06:00:00'))
(Timestamp('2019-04-29 04:00:00'), Timestamp('2019-04-29 06:00:00'))
(Timestamp('2019-04-29 06:00:00'), Timestamp('2019-04-29 08:00:00'))
(Timestamp('2019-04-29 08:00:00'), Timestamp('2019-04-29 11:00:00'))
(Timestamp('2019-04-29 11:00:00'), Timestamp('2019-04-29 13:00:00'))
(Timestamp('2019-04-29 13:00:00'), Timestamp('2019-04-29 15:00:00'))
(Timestamp('2019-04-29 15:00:00'), Timestamp('2019-04-29 17:00:00'))
(Timestamp('2019-04-29 17:00:00'), Timestamp('2019-04-29 19:00:00'))
(Timestamp('2019-04-29 19:00:00'), Timestamp('2019-04-29 22:00:00'))
(Timestamp('2019-04-29 22:00:00'), Timestamp('2019-04-30 00:00:00'))
(Timestamp('2019-04-30 00:00:00'), Timestamp('2019-04-30 02:00:00'))
(Timestamp('2019-04-30 02:00:00'), Timestamp('2019-04-30 04:00:00'))
(Timestamp('2019-04-30 04:00:00'), Timestamp('2019-04-30 06:00:00'))
(Timestamp('2019-04-30 06:00:00'),

In [24]:
import numba as nb
from mp_engine import processJobs_, processJobs
@nb.njit
def func(arr,i):
    col = arr[i]
    mask = np.where(col>0)
    return np.mean(col[mask])

@nb.njit
def njit_getAvgUniqueness(indM):
    # Average uniqueness from indicator matrix
    c=indM.sum(axis=1).reshape(-1,1) # concurrency
    u=np.divide(indM,c) # uniqueness
    avgU = np.zeros(len(u.T)) # avg. uniqueness
    i = 0
    for i in range(len(u.T)):
        avgU[i] = func(u.T,i)
        i+=1
    return avgU

@nb.jit(nopython=False)
def jit_seqBootstrap(indM,sLength=None):
    # Generate a sample via sequential bootstrap
    if sLength is None:sLength=indM.shape[1]
    phi=[]
    while len(phi)<sLength:
        avgU=pd.Series()
        for i in indM:
            indM_=indM[phi+[i]] # reduce indM
            avgU.loc[i]=njit_getAvgUniqueness(indM_.values)[-1]
        prob=avgU/avgU.sum() # draw prob
        phi+=[np.random.choice(indM.columns,p=prob)]
    print("One down")
    return phi
#------------------------



In [25]:
#ind_matrix = fml_lib.getIndMatrix(dollar_bars.index,events['t1'])
#bootstrap_indices = fml_lib.seqBootstrap(ind_matrix)
bootstrap_indices = fml_lib.main_mp(events['t1'], 100, 4)
print(bootstrap_indices)

c:\Python39\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


processing
pool made
variables initialized
